# CMU auto-graded notebook

Before you turn these assignments in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE`, `<FILL IN>`, or "YOUR ANSWER HERE."


---


# CMU Machine Learning with Large Datasets

## Homework 1 - Coding 2: Streaming Naive Bayes


In [ ]:
# Who did you collaborate with on this assignment?
# if no one, collaborators should contain an empty string,
# else list your collaborators below

# collaborators = [""]
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
try:
    collaborators
except:
    raise AssertionError("you did not list your collaborators, if any")

### (1a) Environment Setup

Run the following cell to establish the runtime environment.


In [ ]:
# Ignore this cell if the environment already has the packages

%pip install nose numpy

In [ ]:
# imports that will be used in the notebook -- shouldn't need to import any other libraries

import math
import os
import re
from collections import Counter

import numpy as np
from nose.tools import assert_equal

### (1b) Data Preparation


We use the Reuters Corpus (RCV1), which is a set of news stories split into a hierarchy of categories. There are three file sets. The two data sets with "small" in the name contain smaller subsamples of the full data set. They are provided for debugging and local tests. The final classification task should use the "full" one. Each data set is split into a train and test set, as indicated by the file suffix:

- RCV1.full_train.txt
- RCV1.full_test.txt
- RCV1.small_train.txt
- RCV1.small_test.txt
- RCV1.very_small_train.txt
- RCV1.very_small_test.txt

Download the data using the link provided in the handout, and fill in the corresponding variables with their file paths in the following cell.


In [ ]:
# Uncomment this cell if running on Databricks

# from urllib.parse import urlparse
# from pyspark import SparkFiles

# def get_file(url):
#     parsed_url = urlparse(url)
#     suffix = parsed_url.path.split('/')[-1]
#     sc.addFile(url)
#     path = SparkFiles.get(suffix)

#     return path

# FULL_TRAIN = get_file("https://cmu.box.com/shared/static/g37w2q552qaik60yubrd3aq95szgi7pc")
# FULL_TEST = get_file("https://cmu.box.com/shared/static/305174svzxe3lqm0ps76fvaenhr37inj")
# SMALL_TRAIN = get_file("https://cmu.box.com/shared/static/f1x91jode5ywofu5b6cm9tb6ynx3yoft")
# SMALL_TEST = get_file("https://cmu.box.com/shared/static/0l8y7x3gtix2sjfb1ewzq10c8ayaokdd")
# VERY_SMALL_TRAIN = get_file("https://cmu.box.com/shared/static/xiumqmfdge5muxhs9v2w7ma97hurqtqy")
# VERY_SMALL_TEST = get_file("https://cmu.box.com/shared/static/np2s7tn5wvwuawad2vddygs76tbwt1f5")

In [ ]:
# TODO: Replace <FILL IN> with appropriate file paths
FULL_TRAIN = <FILL IN>
FULL_TEST = <FILL IN>
SMALL_TRAIN = <FILL IN>
SMALL_TEST = <FILL IN>
VERY_SMALL_TRAIN = <FILL IN>
VERY_SMALL_TEST = <FILL IN>

There are multiple class labels per document, meaning that there is more than one correct answer to the question "What kind of news article is this?"

For this assignment, we will ignore all class labels except for those ending in CAT and just be classifying into the top-level nodes of the hierarchy:

- CCAT: Corporate/Industrial
- ECAT: Economics
- GCAT: Government/Social
- MCAT: Markets

DO NOT change the following cell and just run it to set up the CAT labels


In [ ]:
CAT_LABELS = ['CCAT', 'ECAT', 'GCAT', 'MCAT']

The data format is one document per line, with the class label(s) first (comma separated), a tab character, and then the document text.

Run the following cell to take a glance at the first document of the small training data set.


In [ ]:
with open(SMALL_TRAIN, 'r') as f:
    print(f.readline())

### (1c) Data Processing


To count the words, we need to tokenize the document text. In real-world practice, this may involve multiple steps, such as removing stop words and converting text to lowercase, which you learned in HW1: Entity Resolution. For this Naive Bayes part, we simplify the process by splitting only on words.

Run the following cell to define the `tokenization(doc)` function.


In [ ]:
# DO NOT change this function
def tokenizeDoc(doc: str) -> list[str]:
    """
    Convert input document text into tokenization features
    Args:
        doc: document text
    Returns:
        list: a list of tokens
    """
    return re.findall('\\w+', doc)

As the handout instructs, streaming Naive Bayes loads one-line document at a time, use that document to update the classifier statistics, and then discard the document. After loading a line, we need a function to parse the line for classifier training.

Implement `parseDatafileLine(datafileLine)` that takes a line (labels + document text) and return a list of labels and a list of tokens. You need to use the `tokenizeDoc(doc)` function defined above.


In [ ]:
def parseDatafileLine(datafileLine: str):
    """
    Parse a line of the data file to separate labels and document tokens
    Args:
        datafileLine: input string that is a line from the data file
    Returns:
        labels (list), tokens (list)
    """
    # TODO: YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
"""Test parseDatafileLine(datafileLine)"""

line_sample1 = "C15,C151,CCAT\tMcDonald's Corp said Thursday it raised its quarterly dividend 10 percent, to $0.0825 a share from $0.075."
line_sample2 = "C15,C151,CCAT\tSix months to Sept 30, 1996       (in million rupees unless stated)"

assert_equal(parseDatafileLine(line_sample1),
             (['C15', 'C151', 'CCAT'],
              ['McDonald', 's', 'Corp', 'said', 'Thursday', 'it', 'raised', 'its',
               'quarterly', 'dividend', '10', 'percent', 'to', '0', '0825', 'a',
               'share', 'from', '0', '075']))

assert_equal(parseDatafileLine(line_sample2),
             (['C15', 'C151', 'CCAT'],
              ['Six', 'months', 'to', 'Sept', '30', '1996', 'in', 'million',
               'rupees', 'unless', 'stated']))

### 1(d) Training


Now, we are good for training. We will use a dictionary as the model to store the count statistics.

As the handout instructs, the model contains five parts:

- `y`: $(Y=y)$ for each label y the number of training instances of that class
- `ys`: $(Y=*)$ here $*$ means anything, so this is just the total number of training instances
- `y_w`: $(Y=y,W=w)$ number of times token w appears in a document with label y
- `y_ws`: $(Y=y,W=*)$ total number of tokens for documents with label y
- `vocabulary`: track the vocabulary for Laplace smoothing

Recall that Laplace smoothing formula is

$$p(W=w_i|Y=y)=\frac{count(Y=y,W=w_i) + \alpha}{count(Y=y,W=*)+ \alpha|V|}$$
where $|V|$ is the vocabulary.

Implement `nbmodel()` by figuring out the proper variable types for each part and filling in the blank below.


In [ ]:
def nbmodel() -> dict:
    """
    Returns:
        dict storing the required five parts
    """
    # TODO: Replace <FILL IN> with appropriate code
    return {
        'y': <FILL IN>,
        'ys': <FILL IN>,
        'y_w': <FILL IN>,
        'y_ws': <FILL IN>,
        'vocabulary': <FILL IN>
    }

Implement `trainNB(trainfile, model)` that loads one document at a time and uses that document to update the required statistics for the Naive Bayes classifier.

Hint:

1. We only use those lables ending in CAT, which defined in `CAT_LABELS` earlier. Therefore, you need to figure out a way to skip other labels.
2. There are some documents with more than one CAT label. Treat those documents as multiple data instances (that is, add to the counters for
   all labels ending in CAT). For instance, if one line contains CCAT and ECAT, use the same document text twice.
3. It is not necessary to return the model here if you use proper types. Think about Python's mutable vs immutable types.


In [ ]:
def trainNB(trainfile: str, model: dict):
    """
    Update the model in a streaming fashion.
    Args:
        trainfile: file path of the training data
        model: dict of the Naive Bayes classifier model
    """
    with open(trainfile, 'r') as f:
        # Please note here we use f for iteration directly instead of f.readlines().
        # Think about the reasons from the perspective of streaming
        for line in f:
            labels, tokens = parseDatafileLine(line)

            # TODO: YOUR CODE HERE
            raise NotImplementedError()

### 1(e) Test


For each line of documents, your classification code should get the best class $y$ and its log probabilities:

$$ln(p(Y=y))+\sum_{w_i} ln(p(W=w_i|Y=y))$$

Notice that we use the natural logarithm here.

Implement `testNB(testfile, model)` that uses the trained model to classify the test data and return a list of best classes, a list of max log probabilities (**rounding it to 4 decimal places**), and overall accuracy (**rounding it to 4 decimal places**). Please explicitly return in this specified order.


In [ ]:
def testNB(testfile, model):
    """
    Implement Naive Bayes classification
    Args:
        testfile: file path of the test data
        model: dict of the Naive Bayes classifier model
    Returns:
        best_classes, log_probabilities, accuracy
    """
    best_classes = <FILL IN>
    log_probabilities = <FILL IN>

    with open(testfile, 'r') as f:
        for line in f.readlines():
            labels, tokens = parseDatafileLine(line)

            # TODO: YOUR CODE HERE
            raise NotImplementedError()

    return best_classes, log_probabilities, accuracy

In [ ]:
"""DO NOT change this this cell and just run it to use the very small dataset to test your implementations"""

very_small_model = nbmodel()
trainNB(VERY_SMALL_TRAIN, very_small_model)
best_classes, log_probabilities, accuracy = testNB(VERY_SMALL_TEST, very_small_model)

assert_equal(best_classes,
             ['MCAT', 'ECAT', 'CCAT', 'ECAT', 'CCAT', 'CCAT', 'ECAT', 'CCAT'])
assert_equal(log_probabilities,
             [-9893.7804, -3912.8180, -1121.5992, -1610.1660,
              -701.3466, -1453.3430, -2218.3302, -2285.0698])
assert_equal(accuracy, 0.8750)

### 1(f) Full Classification and Deliverable

We are almost there! Let's wrap up this assignment.

Implement your training and test functions on the full dataset to get the full classification results. Please note that you need to define a new model different from the `very_small_model` we have already tested.

Write the full classification results to a file `full_result.txt` (please explicitly use this name). The output format should have one test result per line, and each line should have the format:

$$\text{[Label1, Label2, ...]<tab>Best class<tab>Log prob}$$

where **[Label1, Label2, ...]** are the true labels of the test instance, **Best class** is the class with the maximum log probability, and the last field is the log probability.

The last line of the file should include the accuracy.

Use the following cell to write your code.

Submit both this notebook and `full_result.txt` to Gradescope.


In [ ]:
# * Here is an expected output of very_small_test dataset for your reference
# * You need to write one using the FULL dataset

'''
['C24', 'CCAT', 'M14', 'MCAT']\tMCAT\t-9893.7804
['E51', 'E512', 'ECAT', 'GCAT', 'GDIP']\tECAT\t-3912.8180
['C15', 'C152', 'C18', 'C181', 'CCAT']\tCCAT\t-1121.5992
['GCAT']\tECAT\t-1610.1660
['C13', 'CCAT', 'GCAT', 'GHEA']\tCCAT\t-701.3466
['C13', 'CCAT', 'M11', 'MCAT']\tCCAT\t-1453.3430
['C11', 'C13', 'CCAT', 'E12', 'ECAT', 'M13', 'M132', 'MCAT']\tECAT\t-2218.3302
['C31', 'CCAT']\tCCAT\t-2285.0698
Accuracy: 7/8=0.8750
'''